# Autodiff

<img src="https://raw.githubusercontent.com/google/jax/main/images/jax_logo.svg" alt="JAX Logo" height ="50"/>

JAX is like numpy, but with autodiff. This means that you can compute gradients of functions with respect to their inputs.


<small>Other frameworks like TensorFlow, Pytorch also have autodiff</small>

## Simple Example

### Standart numpy

In [1]:
import numpy as np

def f(x,y,z):
    return (x+y)*z

f(-2,5,-4)

-12

### Same with JAX

In [2]:
import jax.numpy as jnp
from jax import grad

def g(x,y,z):
    return (x+y)*z

print(f'g(-2,5,-4) = {g(-2,5,-4)}')
print(f'grad(g)|(-2.,5.,-4.) = {grad(g)(-2.,5.,-4.)}')

g(-2,5,-4) = -12
grad(g)|(-2.,5.,-4.) = -4.0


In [3]:
### Second order derivatives
def gg(x):
    return x**4

print(f'{gg(2.)}')
ggg = grad(gg)
print(f'{ggg(2.)}')
print(f'{grad(ggg)(2.)}')

16.0
32.0
48.0


## Autodiff of functions (in a coding sense)

Autodiff is not limited to simple functions. You can compute gradients of complicated pythin functions with respect to their inputs.

### Calculating Sums with loops

Let's look at the following function:
$$
f(x) = \sum_{k=1}^{N} \left( \ln(x + k) - \ln(x + k - 1) \right)
$$

The derivative of this function w.r.t. $x$ is (feel free to try to derive it yourself):
$$
f^{\prime}(x) = \frac{1}{x + N} - \frac{1}{x}
$$

Let's implement this function in a naive way and calculate the derivative using JAX.

In [4]:
import jax
import jax.numpy as jnp

def f(x, N):
    result = 0.0
    for k in range(1, N + 1):
        term = jnp.log(x + k) - jnp.log(x + k - 1)
        result += term
    return result

# Derivative respect to x, always uses the first argument
df_dx = jax.grad(f)

# Example values
x_value = 1.0
N_value = 5

df_dx(x_value, N_value) # Should be 1/(1 + N) - 1 = 1/6 - 1 ~ -0.8333333 

Array(-0.8333334, dtype=float32, weak_type=True)

## Custom functions / Idea of gradient calculation

The next examples shows how to JAX calulates the gradient. Every function needs to provide a gradient. For simple functions this is already implemented. Let's define a custom function in which we define the gradient ourselves. We take a look at the ReLU-like function, which is quadratic for positive values and 0 for negative values. We define the function as follows:


$$
\text{qlu}(x) = 
\begin{cases} 
x^2, & \text{if } x > 0 \\
0, & \text{if } x \leq 0 
\end{cases}
$$
Which is continuous everywhere except at 0 but the gradient is not defined at 0. We can define the gradient to be 0.42 at 0. The flowing diagram shows the function and its gradient. 

<center>
    <img src="qlu.svg" alt="QLU Diagram" height="100"/>
</center>


To be used in the "Chain Rule" we need to define the gradient of the function w.r.t. x. This is a function that takes the value of x and the "incomming gradient" grad_y and returns the gradient of the function w.r.t. x. 



In [5]:
from jax import custom_vjp
# Define the custom qlu function and state that, we provide custom gradients. 
# By defining the forward and backward functions.
@custom_vjp
def qlu(x):
    return jnp.maximum(0, x**2)

# Forward function: computes the output and any intermediate values
def qlu_fwd(x):
    y = qlu(x) 
    return y, x  # Pass x to the backward function for gradient handling

# Backward function: provides a custom gradient rule
def qlu_bwd(x, grad_y):
    # Gradient is:
    # 0 for x < 0
    # 1 for x > 0
    # 0.42 at x = 0
    grad_x = jnp.where(x > 0, 2*x*grad_y, 0)  # Grad = x for x > 0
    grad_x = jnp.where(x == 0, grad_y * 0.42, grad_x)  # Grad = 0.42 for x == 0
    return (grad_x,)

# Register forward and backward pass
qlu.defvjp(qlu_fwd, qlu_bwd)

# Test the function and its gradient
x_values = jnp.array([-1.0, 0.0, 2.0])  # Test points
print("Custom ReLU values:", qlu(x_values))

# Compute gradients
grads = jax.grad(lambda x: qlu(x).sum())(x_values)
print("Gradients of Custom ReLU at test points:", grads)

Custom ReLU values: [1. 0. 4.]
Gradients of Custom ReLU at test points: [0.   0.42 4.  ]
